In [1]:
import sqlite3
from datetime import date
import holidays

conn = sqlite3.connect("my_database_project.db")
cur = conn.cursor()

In [2]:
def is_leap_year(year):
    if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
        return True
    return False

In [3]:
#create table for datum 

cur.execute("""
CREATE TABLE IF NOT EXISTS Date_dim (
    DateID INTEGER PRIMARY KEY AUTOINCREMENT,
    Date TEXT NOT NULL,
    year INTEGER NOT NULL,
    month INTEGER NOT NULL,
    day INTEGER NOT NULL,
    day_of_the_week TEXT NOT NULL,
    is_holiday BOOLEAN DEFAULT 0,
    quarter INTEGER NOT NULL
)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS time_dim (
    timeID INTEGER PRIMARY KEY,
    time_of_the_day TEXT NOT NULL 
)
""")
conn.commit()


In [4]:
dates = []
for year in range(2018, 2024):
    berlin_holidays = holidays.Germany(years =year,subdiv='BE')
    for month in range(1, 13):
        for day in range(1, 32):
            if month == 2 and is_leap_year(year) and day == 30:
                break
            elif month == 2 and not is_leap_year(year) and day == 29:
                break
            elif month in (4,6,9,11)  and day == 31:
                break
            elif month in (1,3,5,7,8,10,12) and day == 32:
                break
            
            d = date(year, month, day)  # Year, Month, Day
            formatted_date = d.strftime("%d.%m.%Y")
            is_holiday = d in berlin_holidays

            day_of_week = d.weekday()
            quarter = (month - 1) // 3 + 1
            d_to_save= (formatted_date, year ,month , day, d.weekday(), is_holiday,quarter)
            dates.append(d_to_save)

cur.executemany("INSERT INTO Date_dim(date, year, month, day,day_of_the_week, is_holiday,quarter) VALUES (?,?,?,?,?,?,?)" , dates)
conn.commit()

In [5]:
#check data is well stored 
cur.execute('select max(DateID) from date_dim ').fetchall()

[(2191,)]

In [6]:
time_of_the_day = ''
for i in range (0, 24):
    if i >= 4 and i < 10 :
        time_of_the_day ='morgens'
    if i >= 10 and i < 12:
          time_of_the_day ='vormittags'
    if i == 12 :
        time_of_the_day ='mittags'
    if i > 12 and i < 17:
          time_of_the_day ='nachmittags'
    if i > 17 and i <= 22:
          time_of_the_day ='abends'      
    if i > 22 or i < 4:
      time_of_the_day ='nachts'
   
        
    cur.execute("INSERT INTO time_dim(timeID, time_of_the_day) VALUES  (?, ?)", (i,time_of_the_day))
    conn.commit()

In [7]:
#check data is well stored 
# we can add a Spalte hour as a string '00' or '01' ...
cur.execute('select * from time_dim').fetchall()

[(0, 'nachts'),
 (1, 'nachts'),
 (2, 'nachts'),
 (3, 'nachts'),
 (4, 'morgens'),
 (5, 'morgens'),
 (6, 'morgens'),
 (7, 'morgens'),
 (8, 'morgens'),
 (9, 'morgens'),
 (10, 'vormittags'),
 (11, 'vormittags'),
 (12, 'mittags'),
 (13, 'nachmittags'),
 (14, 'nachmittags'),
 (15, 'nachmittags'),
 (16, 'nachmittags'),
 (17, 'nachmittags'),
 (18, 'abends'),
 (19, 'abends'),
 (20, 'abends'),
 (21, 'abends'),
 (22, 'abends'),
 (23, 'nachts')]

In [8]:
cur.close()
conn.close()